In [1]:
import sys
sys.path.append("../../")

import numpy as np

In [2]:
# Constant Parameters

REGENERATE = True
PICKLE_PREFIX = "pickles/stark_sliders/"

COMMON_ARGS = {
    "temperature_kelvin": 300,
    "energy_gap": 8_000,
    "mode_frequencies": [1200, 100],
    "mode_couplings": [0.7, 2.0],
}

TWO_STATE_ARGS = {
    "transfer_integral": 100,
    "broadening": 200,
    "mode_basis_sets": [20, 200],
}

MLJ_ARGS = {
    "disorder_meV": 0,
    "basis_size": 20,
}

STARK_ARGS = {
    "positive_field_sum_percent": 0.5,
}

In [3]:
from dataclasses import dataclass
import ipywidgets as widgets

@dataclass
class DefaultMinMaxNum:
    default: float
    min: float
    max: float
    num: int
    
    def to_range(self):
        return np.linspace(self.min, self.max, self.num)
    
    def to_slider(self):
        step = (self.max - self.min) / (self.num - 1)
        return widgets.FloatSlider(min=self.min, max=self.max, step=step, value=self.default)

# Slider Parameters
SLIDER_ARGS = {
    "g1": DefaultMinMaxNum(0.7, 0.7, 0.7, 1),
    "g2": DefaultMinMaxNum(2, 8, 2, 4),
    "positive_field_strength": DefaultMinMaxNum(0.01, 0.01, 0.01, 1),
    "field_delta_dipole": DefaultMinMaxNum(38, 0, 38, 4),
    "field_delta_polarizability": DefaultMinMaxNum(0, 0, 1_000, 4),
}

In [4]:
from itertools import product
import inspect

from quantumspectra_2024.models import StarkModel, TwoStateModel, MLJModel
from plot_utils import save_file, open_file

# PREPROCESS SPECTRA

two_state_valids = list(inspect.signature(TwoStateModel).parameters)
mlj_valids = list(inspect.signature(MLJModel).parameters)
stark_valids = list(inspect.signature(StarkModel).parameters)

def compute_spectra(slider_params):
    all_args = {**COMMON_ARGS, **slider_params, **TWO_STATE_ARGS, **MLJ_ARGS, **STARK_ARGS}
    
    two_state_args = {k: v for k, v in all_args.items() if k in two_state_valids}
    mlj_args = {k: v for k, v in all_args.items() if k in mlj_valids}
    stark_args = {k: v for k, v in all_args.items() if k in stark_valids}
    
    two_state_args["mode_couplings"] = [slider_params["g1"], slider_params["g2"]]
    mlj_args["mode_couplings"] = [slider_params["g1"], slider_params["g2"]]
    
    two_state_model = TwoStateModel(**two_state_args)
    mlj_model = MLJModel(**mlj_args)
    
    two_state_stark_model = StarkModel(neutral_submodel=two_state_model, **stark_args)
    mlj_stark_model = StarkModel(neutral_submodel=mlj_model, **stark_args)
    
    return (
        two_state_model.get_absorption(), 
        mlj_model.get_absorption(), 
        two_state_stark_model.get_absorption(), 
        mlj_stark_model.get_absorption(),
    )

def get_spectra(slider_params):
    if REGENERATE:
        ts_abs, mlj_abs, ts_stark_abs, mlj_stark_abs = compute_spectra(slider_params)
        save_file(tuple(ts_abs, mlj_abs, ts_stark_abs, mlj_stark_abs), f"{PICKLE_PREFIX}{slider_params}.pkl")
    else:
        ts_abs, mlj_abs, ts_stark_abs, mlj_stark_abs = open_file(f"{PICKLE_PREFIX}{slider_params}.pkl")
        
    return ts_abs, mlj_abs, ts_stark_abs, mlj_stark_abs
        
        
if REGENERATE:
    # matches arrays of valid values to parameter names
    range_dict = {k: v.to_range() for k, v in SLIDER_ARGS.items()}

    # generate all combinations of parameter values
    param_values = list(product(*range_dict.values()))
    labeled_param_values = [dict(zip(range_dict.keys(), p)) for p in param_values]
    
    for slider_params in labeled_param_values:
        compute_spectra(slider_params)